In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/Data

/content/drive/MyDrive/Data


In [ ]:
import pandas as pd
non_PRC = pd.read_csv('looking_back/50%/look50.csv')
PRC = pd.read_csv('looking_back/50%/look_par50.csv')
clinical_assessment = pd.read_excel('Clinical Assessment.xlsx')

In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV,cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
import warnings
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt

In [ ]:
score = clinical_assessment.iloc[:,9]

In [ ]:
score = score.dropna()
del score[0]
del score[1]
del score[9]
del score[18]

In [ ]:
y = []
for value in score:
    if value >= 45:
        y.append(1)
    else:
        y.append(0)
print(y)

[1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0]


## Non-Paretic Side

In [ ]:
df_non = pd.DataFrame(non_PRC)
y = pd.DataFrame(y)
non = pd.concat([y,df_non],axis=1)
colname = list(non.columns)
colname[0] = "score"
non.columns = colname
non.columns

Index(['score', 'Unnamed: 0', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7',
       'PC8', 'PC9', 'PC10', 'PC11', 'PC12', 'PC13', 'PC14', 'PC15', 'PC16',
       'PC17', 'PC18', 'PC19', 'PC20', 'PC21', 'PC22', 'PC23', 'PC24', 'PC25',
       'PC26', 'PC27', 'PC28', 'PC29', 'PC30', 'PC31', 'PC32', 'PC33', 'PC34',
       'PC35', 'PC36', 'PC37', 'PC38', 'PC39', 'PC40', 'PC41'],
      dtype='object')

In [ ]:
X = non.loc[:,['PC5','PC20','PC27','PC33','PC41']]
y = non['score']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=123, stratify=y)

### Logistic Regression

In [ ]:
lr_model = LogisticRegression()

# 선택된 특성으로 모델 학습
lr_model.fit(X_train, y_train)

accuracy_scores = cross_val_score(lr_model, X_test, y_test, scoring='accuracy', cv=5)
precision_scores = cross_val_score(lr_model, X_test, y_test, scoring='precision', cv=5)
f1_scores = cross_val_score(lr_model, X_test, y_test, scoring='f1', cv=5)
auc_scores = cross_val_score(lr_model, X_test, y_test, scoring='roc_auc', cv=5)


print("Average Accuracy:", np.mean(np.round(accuracy_scores,3)))
print("Average Precision:", np.mean(np.round(precision_scores,3)))
print("Average F1-score:", np.mean(np.round(f1_scores,3)))
print("Average AUC:", np.mean(np.round(auc_scores,3)))

Confusion Matrix:
 [[ 1  4]
 [ 0 12]]
Test Accuracy: 0.7647058823529411
f1-score: 0.8571428571428571
AUC: 0.6
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.20      0.33         5
           1       0.75      1.00      0.86        12

    accuracy                           0.76        17
   macro avg       0.88      0.60      0.60        17
weighted avg       0.82      0.76      0.70        17



### RandomForest

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier


param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}


grid_search = GridSearchCV(RandomForestClassifier(random_state=123), param_grid, cv=5, scoring='accuracy')


grid_search.fit(X_train, y_train)


# 최적의 모델과 하이퍼파라미터 출력
best_rf_model = grid_search.best_estimator_

accuracy_scores = cross_val_score(best_rf_model, X_test, y_test, scoring='accuracy', cv=5)
precision_scores = cross_val_score(best_rf_model, X_test, y_test, scoring='precision', cv=5)
f1_scores = cross_val_score(best_rf_model, X_test, y_test, scoring='f1', cv=5)
auc_scores = cross_val_score(best_rf_model, X_test, y_test, scoring='roc_auc', cv=5)


print("Average Accuracy:", np.mean(np.round(accuracy_scores,3)))
print("Average Precision:", np.mean(np.round(precision_scores,3)))
print("Average F1-score:", np.mean(np.round(f1_scores,3)))
print("Average AUC:", np.mean(np.round(auc_scores,3)))

Best Random Forest Model: RandomForestClassifier(min_samples_leaf=4, random_state=123)
Best Hyperparameters: {'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 100}
Confusion Matrix:
 [[ 0  5]
 [ 0 12]]
Test Accuracy with Best Model: 0.7058823529411765
f1-score with Best Model: 0.8275862068965517
AUC with Best Model: 0.5
Classification Report with Best Model:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       0.71      1.00      0.83        12

    accuracy                           0.71        17
   macro avg       0.35      0.50      0.41        17
weighted avg       0.50      0.71      0.58        17



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### SVM Linear

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear'],
    'degree':[2,3,4]
}


grid_search = GridSearchCV(SVC(random_state=123), param_grid, cv=5, scoring='accuracy')


grid_search.fit(X_train, y_train)


best_svm_model = grid_search.best_estimator_
accuracy_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='accuracy', cv=5)
precision_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='precision', cv=5)
f1_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='f1', cv=5)
auc_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='roc_auc', cv=5)

# Display the average performance metrics over the 5 folds
print("Average Accuracy:", np.mean(np.round(accuracy_scores,3)))
print("Average Precision:", np.mean(np.round(precision_scores,3)))
print("Average F1-score:", np.mean(np.round(f1_scores,3)))
print("Average AUC:", np.mean(np.round(auc_scores,3)))

Best SVM Model: SVC(C=0.1, degree=2, kernel='linear', random_state=123)
Best Hyperparameters: {'C': 0.1, 'degree': 2, 'kernel': 'linear'}
Confusion Matrix:
 [[ 0  5]
 [ 0 12]]
Test Accuracy with Best Model: 0.7058823529411765
f1-score with Best Model: 0.8275862068965517
AUC with Best Model: 0.8500000000000001
Classification Report with Best Model:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       0.71      1.00      0.83        12

    accuracy                           0.71        17
   macro avg       0.35      0.50      0.41        17
weighted avg       0.50      0.71      0.58        17



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### SVM Poly

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['poly'],
    'degree': [2, 3, 4],
}


grid_search = GridSearchCV(SVC(random_state=123), param_grid, cv=5, scoring='accuracy')


grid_search.fit(X_train, y_train)


best_svm_model = grid_search.best_estimator_
accuracy_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='accuracy', cv=5)
precision_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='precision', cv=5)
f1_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='f1', cv=5)
auc_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='roc_auc', cv=5)

# Display the average performance metrics over the 5 folds
print("Average Accuracy:", np.mean(np.round(accuracy_scores,3)))
print("Average Precision:", np.mean(np.round(precision_scores,3)))
print("Average F1-score:", np.mean(np.round(f1_scores,3)))
print("Average AUC:", np.mean(np.round(auc_scores,3)))

Best SVM Model: SVC(C=0.1, degree=2, kernel='poly', random_state=123)
Best Hyperparameters: {'C': 0.1, 'degree': 2, 'kernel': 'poly'}
Confusion Matrix:
 [[ 0  5]
 [ 0 12]]
Test Accuracy with Best Model: 0.7058823529411765
f1-score with Best Model: 0.8275862068965517
AUC with Best Model: 0.5333333333333333
Classification Report with Best Model:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       0.71      1.00      0.83        12

    accuracy                           0.71        17
   macro avg       0.35      0.50      0.41        17
weighted avg       0.50      0.71      0.58        17



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### XGBoost

In [ ]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb


param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.1, 0.2],
}


grid_search = GridSearchCV(xgb.XGBClassifier(random_state=123), param_grid, cv=5, scoring='accuracy')


grid_search.fit(X_train, y_train)


best_xgb_model = grid_search.best_estimator_
accuracy_scores = cross_val_score(best_xgb_model, X_test, y_test, scoring='accuracy', cv=5)
precision_scores = cross_val_score(best_xgb_model, X_test, y_test, scoring='precision', cv=5)
f1_scores = cross_val_score(best_xgb_model, X_test, y_test, scoring='f1', cv=5)
auc_scores = cross_val_score(best_xgb_model, X_test, y_test, scoring='roc_auc', cv=5)

# Display the average performance metrics over the 5 folds
print("Average Accuracy:", np.mean(np.round(accuracy_scores,3)))
print("Average Precision:", np.mean(np.round(precision_scores,3)))
print("Average F1-score:", np.mean(np.round(f1_scores,3)))
print("Average AUC:", np.mean(np.round(auc_scores,3)))

Best XGBoost Model: XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=123, ...)
Best Hyperparameters: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100}
Confusion Matrix:
 [[ 0  5]
 [ 0 12]]
Test Accuracy with Best Model: 0.7058823529411765
f1-score with Best Model: 0.8275862068965517
AUC with Be

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Paretic Side

In [ ]:
df_par = pd.DataFrame(PRC)
y = pd.DataFrame(y)
par = pd.concat([y,df_par],axis=1)
colname = list(par.columns)
colname[0] = "score"

In [ ]:
import random
random.seed(1)
X = par.loc[:,['PC2','PC5','PC23','PC28','PC34','PC36','PC39']]
y = par['score']
X_train , X_test , y_train , y_test  = train_test_split(X,y,test_size = 0.4,random_state=123,stratify=y)

### Logistic Regression

In [ ]:
lr_model = LogisticRegression()

# 선택된 특성으로 모델 학습
lr_model.fit(X_train, y_train)

accuracy_scores = cross_val_score(lr_model, X_test, y_test, scoring='accuracy', cv=5)
precision_scores = cross_val_score(lr_model, X_test, y_test, scoring='precision', cv=5)
f1_scores = cross_val_score(lr_model, X_test, y_test, scoring='f1', cv=5)
auc_scores = cross_val_score(lr_model, X_test, y_test, scoring='roc_auc', cv=5)


print("Average Accuracy:", np.mean(np.round(accuracy_scores,3)))
print("Average Precision:", np.mean(np.round(precision_scores,3)))
print("Average F1-score:", np.mean(np.round(f1_scores,3)))
print("Average AUC:", np.mean(np.round(auc_scores,3)))

Confusion Matrix:
 [[ 0  5]
 [ 1 11]]
Test Accuracy: 0.6470588235294118
f1-score: 0.7857142857142857
AUC: 0.4583333333333333
Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       0.69      0.92      0.79        12

    accuracy                           0.65        17
   macro avg       0.34      0.46      0.39        17
weighted avg       0.49      0.65      0.55        17



### RandomForest

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}


grid_search = GridSearchCV(RandomForestClassifier(random_state=123), param_grid, cv=5, scoring='accuracy')


grid_search.fit(X_train, y_train)

# 최적의 모델과 하이퍼파라미터 출력
best_rf_model = grid_search.best_estimator_

accuracy_scores = cross_val_score(best_rf_model, X_test, y_test, scoring='accuracy', cv=5)
precision_scores = cross_val_score(best_rf_model, X_test, y_test, scoring='precision', cv=5)
f1_scores = cross_val_score(best_rf_model, X_test, y_test, scoring='f1', cv=5)
auc_scores = cross_val_score(best_rf_model, X_test, y_test, scoring='roc_auc', cv=5)

# Display the average performance metrics over the 5 folds
print("Average Accuracy:", np.mean(np.round(accuracy_scores,3)))
print("Average Precision:", np.mean(np.round(precision_scores,3)))
print("Average F1-score:", np.mean(np.round(f1_scores,3)))
print("Average AUC:", np.mean(np.round(auc_scores,3)))

Best Random Forest Model: RandomForestClassifier(min_samples_leaf=4, random_state=123)
Best Hyperparameters: {'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 100}
Confusion Matrix:
 [[ 0  5]
 [ 0 12]]
Test Accuracy with Best Model: 0.7058823529411765
F1-score with Best Model: 0.8275862068965517
AUC with Best Model: 0.5
Classification Report with Best Model:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       0.71      1.00      0.83        12

    accuracy                           0.71        17
   macro avg       0.35      0.50      0.41        17
weighted avg       0.50      0.71      0.58        17



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### SVM Linear

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear'],
    'degree':[2,3,4]
}


grid_search = GridSearchCV(SVC(random_state=123), param_grid, cv=5, scoring='accuracy')


grid_search.fit(X_train, y_train)


best_svm_model = grid_search.best_estimator_
accuracy_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='accuracy', cv=5)
precision_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='precision', cv=5)
f1_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='f1', cv=5)
auc_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='roc_auc', cv=5)

# Display the average performance metrics over the 5 folds
print("Average Accuracy:", np.mean(np.round(accuracy_scores,3)))
print("Average Precision:", np.mean(np.round(precision_scores,3)))
print("Average F1-score:", np.mean(np.round(f1_scores,3)))
print("Average AUC:", np.mean(np.round(auc_scores,3)))

Best SVM Model: SVC(C=0.1, degree=2, kernel='linear', random_state=123)
Best Hyperparameters: {'C': 0.1, 'degree': 2, 'kernel': 'linear'}
Confusion Matrix:
 [[ 0  5]
 [ 0 12]]
Test Accuracy with Best Model: 0.7058823529411765
F1-score with Best Model: 0.8275862068965517
AUC with Best Model: 0.6166666666666667
Classification Report with Best Model:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       0.71      1.00      0.83        12

    accuracy                           0.71        17
   macro avg       0.35      0.50      0.41        17
weighted avg       0.50      0.71      0.58        17



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### SVM Poly

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['poly'],
    'degree': [2, 3, 4],
}


grid_search = GridSearchCV(SVC(random_state=123), param_grid, cv=5, scoring='accuracy')


grid_search.fit(X_train, y_train)


best_svm_model = grid_search.best_estimator_
accuracy_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='accuracy', cv=5)
precision_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='precision', cv=5)
f1_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='f1', cv=5)
auc_scores = cross_val_score(best_svm_model, X_test, y_test, scoring='roc_auc', cv=5)

# Display the average performance metrics over the 2 folds
print("Average Accuracy:", np.mean(np.round(accuracy_scores,3)))
print("Average Precision:", np.mean(np.round(precision_scores,3)))
print("Average F1-score:", np.mean(np.round(f1_scores,3)))
print("Average AUC:", np.mean(np.round(auc_scores,3)))

Best SVM Model: SVC(C=0.1, degree=2, kernel='poly', random_state=123)
Best Hyperparameters: {'C': 0.1, 'degree': 2, 'kernel': 'poly'}
Confusion Matrix:
 [[ 0  5]
 [ 0 12]]
Test Accuracy with Best Model: 0.7058823529411765
F1-score with Best Model: 0.8275862068965517
AUC with Best Model: 0.6166666666666666
Classification Report with Best Model:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       0.71      1.00      0.83        12

    accuracy                           0.71        17
   macro avg       0.35      0.50      0.41        17
weighted avg       0.50      0.71      0.58        17



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### XGBoost

In [ ]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb


param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.1, 0.5],
}


grid_search = GridSearchCV(xgb.XGBClassifier(random_state=123), param_grid, cv=5, scoring='accuracy')


grid_search.fit(X_train, y_train)


best_xgb_model = grid_search.best_estimator_
accuracy_scores = cross_val_score(best_xgb_model, X_test, y_test, scoring='accuracy', cv=5)
precision_scores = cross_val_score(best_xgb_model, X_test, y_test, scoring='precision', cv=5)
f1_scores = cross_val_score(best_xgb_model, X_test, y_test, scoring='f1', cv=5)
auc_scores = cross_val_score(best_xgb_model, X_test, y_test, scoring='roc_auc', cv=5)

# Display the average performance metrics over the 2 folds
print("Average Accuracy:", np.mean(np.round(accuracy_scores,3)))
print("Average Precision:", np.mean(np.round(precision_scores,3)))
print("Average F1-score:", np.mean(np.round(f1_scores,3)))
print("Average AUC:", np.mean(np.round(auc_scores,3)))

Best XGBoost Model: XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=123, ...)
Best Hyperparameters: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}
Confusion Matrix:
 [[1 4]
 [4 8]]
Test Accuracy with Best Model: 0.5294117647058824
F1-score with Best Model: 0.6666666666666666
AUC with Best Mod